# [Clustering4Ever](https://github.com/Clustering4Ever/Clustering4Ever) by [LIPN](https://lipn.univ-paris13.fr/) [A3](https://lipn.univ-paris13.fr/accueil/equipe/a3/) team

In [ ]:
%%classpath add mvn
org.clustering4ever clustering4ever_2.11 0.9.6

In [ ]:
%%classpath add mvn
org.apache.spark spark-core_2.11 2.4.3

# Clusterwise PLS

In [ ]:
import org.clustering4ever.spark.clustering.clusterwise.{Clusterwise, ClusterwiseCore, PLS, ClusterwiseModel}
import scala.collection.{mutable, immutable}

In [ ]:
import org.apache.spark.SparkContext
import org.apache.spark.SparkContext._
import org.apache.spark.SparkConf

val conf = new SparkConf().setAppName("Simple Application").set("spark.ui.enabled", "false")
val sc = new SparkContext("local[4]", "Simple Application", conf)

In [ ]:
%%bash
wget -P /tmp/ http://www.clustering4ever.org/Datasets/Clusterwise/simdata.csv

In [ ]:
val dataXY = scala.io.Source.fromFile("/tmp/simdata.csv")
  .getLines
  .toSeq
  .map(_.split(',').map(_.toDouble))
  .map( a => (a.take(10), a.takeRight(3)) )
  .zipWithIndex
  .map{ case(xy,idx) => (idx, xy) }

## Run a full Clusterwise
### Parameters

In [ ]:
val init = 20
val nbCV = 10
val g = 4
val h = 2
val sizeBloc = 10
val kPred = 25
val nbLaunch = 1
val standardization = true
val withY = false

In [ ]:
val t0 = System.currentTimeMillis
val (sqrmseCalAndVal, _) = Clusterwise.run(sc, dataXY, g, h, nbCV, init, kPred, withY, standardization, sizeBloc)
val t1 = System.currentTimeMillis

(t1 - t0) / 1000D

In [ ]:
sqrmseCalAndVal

## Run one Initialization of Clusterwise

In [ ]:
val (dsPerClass, predFitted, coIntercept, coXYcoef, critReg, mapsRegCrit, classedReg) = ClusterwiseCore.plsPerDot(dataXY, h, g)

## Run one PLS regression

In [ ]:
val inputX = mutable.ArrayBuffer(dataXY.map{ case (id, (x, y)) => (id, x) }:_*)
val inputY = mutable.ArrayBuffer(dataXY.map{ case (id, (x, y)) => y }:_*)

PLS.runPLS(inputX, inputY, 2)._2